Here we create base class for deepar model

In [20]:
import torch
import torch.nn as nn
import pandas as pd

from torch import Tensor, log, exp

class DeepAR(nn.Module):
    def __init__(
            self,
            batch_size: int,
            hidden_size: int,
            input_size: int,
            likelihood: str = 'normal',
            device: str = 'cpu'
    ):
        """
        This class instantiates DeepAR.

        :param batch_size: size of the batch
        :param hidden_size: number of features in hidden state of rnn cell
        :param inout_size: number of expected features in the input tensor
        :param likelihood: desired likelihood
        :param device: device to calculate on
        """
        super(DeepAR, self).__init__()
        # here we initialize hidden states
        self._h_0 = torch.zeros((batch_size, hidden_size), device=device)
        self._c_0 = torch.zeros((batch_size, hidden_size), device=device)
        self._likelihood = likelihood
        self._device = device

        # here we create base architecture of LSTM cell
        self._lstm_cell = nn.LSTMCell(input_size=input_size, hidden_size=hidden_size)

    @property
    def h_0(self):
        return self._h_0

    @property
    def c_0(self):
        return self._c_0

    @property
    def likelihood(self):
        return self._likelihood

    @property
    def device(self):
        return self._device

    @property
    def lstm_cell(self):
        return self._lstm_cell

    def forward(
            self,
            input_tensor: Tensor # [batch_size, seq_len, input_size]
    ):
        """
        Forward method of our model
        :param input_tensor: input tensor [batch_size, seq_len, input_size]
        :return: output_tensor: output tensor contains parameters or desired distribution [batch_size, seq_len, num_pars]
        """
        batch_size, hidden_size = self.h_0.shape[0], self.h_0.shape[1]
        input_size, seq_len = input_tensor.shape[2], input_tensor.shape[1]
        num_pars = 2
        if self.likelihood == 'normal':
            output_tensor = torch.zeros((batch_size, seq_len, num_pars), device=self.device)
        else:
            raise NotImplementedError('this likelihood not yet implemented, gl&hf')
        # here we iterate through all cells (seq_len)
        num_cells = input_tensor.shape[1]

        if self.likelihood == 'normal':
            for cell_index in range(num_cells):
                self._h_0, self._c_0 = self.lstm_cell(input_tensor[:, cell_index, :], (self.h_0, self.c_0))
                mean = nn.Linear(in_features=hidden_size, out_features=1)(self.h_0)
                variance = log(1 + exp(nn.Linear(in_features=hidden_size, out_features=1)(self.h_0)))
                output_tensor[:, cell_index, 0] = torch.flatten(mean)
                output_tensor[:, cell_index, 1] = torch.flatten(variance)
        else:
            raise NotImplementedError('this likelihood not yet implemented, gl&hf')

        return output_tensor

In [21]:
batch_size, seq_len, input_size, hidden_size = 2, 3, 4, 5
model = DeepAR(input_size=input_size, batch_size=batch_size, hidden_size=hidden_size)
input_tensor = torch.zeros((batch_size, seq_len, input_size))
output_tensor = model(input_tensor)
output_tensor

tensor([[[-0.2897,  0.8674],
         [ 0.2523,  0.7870],
         [-0.2335,  0.5151]],

        [[-0.2897,  0.8674],
         [ 0.2523,  0.7870],
         [-0.2335,  0.5151]]], grad_fn=<CopySlices>)